# Model construction

In [1]:
import sys
sys.path.append("../../.")

NThreads = 16

In [2]:
from Construction.Function import Function
from Construction.Components import SBox, addKey, ID, get_IDn, XOR, FakeComponent
from Construction.CompoundFunction import CompoundFunction, INPUT_ID, OUTPUT_ID
from Construction.IteratedCipher import construct_iterated_cipher
from Modelling.PropModels import UT_matrix
from itertools import product

In [3]:
# components
PRESENT_sbox = SBox(4, 4, [0xC, 5, 6, 0xB, 9, 0, 0xA, 0xD, 3, 0xE, 0xF, 8, 4, 7, 1, 2])
PRESENT_bitpermutation = [0, 16, 32, 48, 1, 17, 33, 49, 2, 18, 34, 50, 3, 19, 35, 51, 4, 20, 36, 52, 5, 21, 37, 53, 6, 22, 38, 54, 7, 23, 39, 55, 8, 24, 40, 56, 9, 25, 41, 57, 10, 26, 42, 58, 11, 27, 43, 59, 12, 28, 44, 60, 13, 29, 45, 61, 14, 30, 46, 62, 15, 31, 47, 63]

In [4]:
# construct round function
PRESENT_roundfunction = CompoundFunction(64, 64)
ids = []
# add sboxes
for _ in range(16):
    ids.append(PRESENT_roundfunction.add_component(PRESENT_sbox))
# connect input to components
for i in range(64):
    PRESENT_roundfunction.connect_components(INPUT_ID, i, ids[i//4], i%4)
# connect components to output
for i in range(64):
    PRESENT_roundfunction.connect_components(ids[i//4], i%4, OUTPUT_ID, PRESENT_bitpermutation[i])
    
key_addition = CompoundFunction(64, 64)
for i in range(64):
    xorid = key_addition.add_component(XOR)
    key_addition.connect_components(INPUT_ID, i, xorid, 0)
    key_addition.connect_to_key(xorid, 1)
    key_addition.connect_components(xorid, 0, OUTPUT_ID, i)

In [5]:
def get_custom_key_add(m):
    key_addition = CompoundFunction(64, 64)
    for i in range(64):
        if (m >> i) & 1:
            xorid = key_addition.add_component(XOR)
            key_addition.connect_components(INPUT_ID, i, xorid, 0)
            key_addition.connect_to_key(xorid, 1)
            key_addition.connect_components(xorid, 0, OUTPUT_ID, i)
        else:
            cid = key_addition.add_component(ID)
            key_addition.connect_components(INPUT_ID, i, cid, 0)
            key_addition.connect_components(cid, 0, OUTPUT_ID, i)
    return key_addition
    
def get_short_key_add(n):
    key_addition = CompoundFunction(n, n)
    for i in range(n):
        xorid = key_addition.add_component(XOR)
        key_addition.connect_components(INPUT_ID, i, xorid, 0)
        key_addition.connect_to_key(xorid, 1)
        key_addition.connect_components(xorid, 0, OUTPUT_ID, i)
    return key_addition

In [6]:
from functools import partial

def bit_permutation(u):
    res = 0
    for i in range(64):
        res |= ((u>>i) & 1)<<PRESENT_bitpermutation[i]
    return res

def invert_bit_permutation(v):
    res = 0
    for i, j in enumerate(PRESENT_bitpermutation):
        res |= ((v>>j) & 1)<<i
    return res

def cor_key_add(u, v, k, m=2**64-1):
    u_ = 0
    c = 0
    for i in range(64):
        if (m >> i) & 1 == 1:
            u_ |= ((u >> i) & 1) << c
            c += 1
    return (-2)**((k & u_).bit_count())

def cor_short_key_add(u, v, k, n):
    return (-2)**((k & u).bit_count())

def cor_sbox(u, v, k, UTm, m=2**64-1):
    vinv = invert_bit_permutation(v)
    correlation = 1
    for i in range(16):
        correlation *= UTm[((vinv & m)>>(4*i))&15, ((u & m)>>(4*i))&15]
    return correlation

def custom_cor_sbox(u, v, k, UTms, custom_bit_perm):
    vinv = 0
    for i, j in enumerate(custom_bit_perm):
        vinv |= ((v>>j) & 1)<<i
    correlation = 1
    for UTm, n, m in UTms:
        correlation *= UTm[vinv & (2**m-1), u & (2**n-1)]
        u >>= n
        vinv >>= m
    return correlation

cor_sbox_init = partial(cor_sbox, UTm = UT_matrix(PRESENT_sbox, 4, 4))
print(UT_matrix(PRESENT_sbox, 4, 4))

[[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  1 -2 -1  2  1 -2  0  0 -2  4  2 -4]
 [ 0  0  1  0  0  0  0 -1  1  0 -1 -1 -1  1  0  2]
 [ 0  0  0  1  0  0  0 -1  1 -1  0 -1 -1  2  0  0]
 [ 1  0  0 -1 -1  0  0  2 -1  1  1 -1  2 -1 -2  0]
 [ 0  1  0 -1  0 -1  0  2  0 -1  1  0  0  2 -1 -2]
 [ 0  0  1 -1  0  0 -1  1  0  1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0  1 -1  0]
 [ 1 -1 -1  2  0  0  1 -1 -1  2  2 -3  0 -1 -2  2]
 [ 0  0  0  1  1 -1 -1  1  0  0  1 -2 -1  1  0  0]
 [ 0  0  0  1  0  0  1 -2  0  1  1 -3  0 -1 -2  4]
 [ 0  0  0  1  0  0  0 -1  0  0  1 -2  0  0 -1  2]
 [ 1 -1 -1  1 -1  1  1  0 -1  2  2 -3  1 -2 -2  2]
 [ 0  0  0  0  0  0  0  1  0  0  1 -1  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  1  1 -2  0 -1 -1  2]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0  0 -1  1]]


In [7]:
from pysat.formula import IDPool
from Modelling.PropModels import to_ord
import numpy as np
from LogicOptimisation.QMC import QMC_optimise_CNF
from bitarrays import bitarray
from bitarrays.bitarray import BitArray
from bitarrays.bitset import BitSet
from math import inf

def UT_matrix_multiply_out_precursor_set(u):
    M = UT_matrix(PRESENT_sbox, 4, 4)
    N = np.zeros((16, 2**(4-u.bit_count())), dtype='longlong')
    c = 0
    for x in range(16):
        if x & u == 0:
            v = np.ones((1, 1), dtype='longlong')
            for i in range(4):
                if ((u >> i) & 1):
                    v = np.kron(np.asarray([[2], [1]], dtype='longlong'), v)
                elif ((x >> i) & 1):
                    v = np.kron(np.asarray([[0], [1]], dtype='longlong'), v)
                else:
                    v = np.kron(np.asarray([[1], [0]], dtype='longlong'), v)
            N[:, c] = (M @ v)[:, 0]
            c += 1
    return N

def get_custom_model(M, threads = NThreads):
    N = np.vectorize(to_ord)(M)
    n_extra_vars = int(np.max(np.ma.masked_invalid(N)))
    m = BitSet(M.shape[0]*M.shape[1]*2**n_extra_vars)
    for j in range(M.shape[0]):
        for i in range(M.shape[1]):
            if N[j, i] != inf:
                m.set((2**int(N[j, i])-1)*M.shape[0]*M.shape[1] + j*M.shape[1] + i)

    dont_care = BitSet(M.shape[0]*M.shape[1]*2**n_extra_vars)
    return QMC_optimise_CNF(m, dont_care, threads=threads), n_extra_vars

multiplied_out_components = {}
def multiply_out_precursor_set(r, u):
    global multiplied_out_components
    roundfs, cor_evals = [key_addition, PRESENT_roundfunction]*r + [key_addition], [cor_key_add, cor_sbox_init]*r + [cor_key_add]
    for n in range(len(roundfs)):
        if roundfs[n] is key_addition:
            roundfs[n] = get_short_key_add(64 - u.bit_count())
            cor_evals[n] = partial(cor_short_key_add, n=64 - u.bit_count())
        else:
            v = 0
            for i in range(64):
                if ((u >> (4*(i//4))) & 0xf) == 0xf:
                    v |= 1<<i
            local2global_bit_map = []
            for i in range(64):
                if (v >> i) & 1 == 0:
                    local2global_bit_map.append(PRESENT_bitpermutation[i])
            used_global_bits = sorted(local2global_bit_map)
            local_bit_perm = [used_global_bits.index(local2global_bit_map[i]) for i in range(len(local2global_bit_map))]
            f = CompoundFunction(64-u.bit_count(), 64-v.bit_count())
            UTms= []
            offset_input, offset_output = 0, 0
            for i in range(16):
                ui =  (u >> 4*i) & 0xf
                if ui != 0xf:
                    M = UT_matrix_multiply_out_precursor_set(ui)
                    UTms.append((M, 4-ui.bit_count(), 4))
                    if ui not in multiplied_out_components:
                        multiplied_out_components[ui] = FakeComponent(4 - ui.bit_count(), 4, *get_custom_model(M))
                    fid = f.add_component(multiplied_out_components[ui])
                    for j in range(4 - ui.bit_count()):
                        f.connect_components(INPUT_ID, j+offset_input, fid, j)
                    for j in range(4):
                        f.connect_components(fid, j, OUTPUT_ID, local_bit_perm[j+offset_output])
                    offset_input += 4 - ui.bit_count()
                    offset_output += 4
            roundfs[n] = f
            cor_evals[n] = partial(custom_cor_sbox, UTms = UTms, custom_bit_perm = local_bit_perm)
            u = bit_permutation(v) 
        if u == 0:
            break
    return roundfs, cor_evals

In [8]:
from Modelling.Search import search_simple_properties_mod_2
from Modelling.Trails import contains_nonzero_trail
from functools import partial
from Modelling.UT_Trails import get_divisibility_no_trail, get_divisibility_no_key_dependent_trail, get_divisibility_no_key_dependence, get_divisibility_of_key_dependence_no_trail
from time import time
from pysat.formula import IDPool
from pysat.card import CardEnc, EncType
from Modelling.ExactComputation import compute_exact_correlation_mod, compute_exact_correlation_mod_var
from functools import reduce
from time import time
from multiprocessing.pool import Pool
from functools import partial

# Analysing properties

## 4 round property
input set is `0xf`

In [9]:
r, u = 3, bit_permutation(0xf)
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 2 0.47995734214782715
1 2 0.4055936336517334
2 2 0.3739333152770996
3 2 0.37745070457458496
4 2 0.37430381774902344
5 2 0.37435245513916016
6 2 0.40750885009765625
7 2 0.3757517337799072
8 2 0.37734198570251465
9 2 0.37669897079467773
10 2 0.377352237701416
11 2 0.40819358825683594
12 2 0.37549638748168945
13 2 0.37894606590270996
14 2 0.37750911712646484
15 2 0.40736913681030273
16 1 0.22021889686584473
17 1 0.2516648769378662
18 1 0.25035595893859863
19 1 0.25050926208496094
20 1 0.2202739715576172
21 1 0.24806618690490723
22 1 0.2504920959472656
23 1 0.2500286102294922
24 1 0.2496330738067627
25 1 0.2216649055480957
26 1 0.24988675117492676
27 1 0.2513554096221924
28 1 0.2513720989227295
29 1 0.2487039566040039
30 1 0.2208113670349121
31 1 0.2485952377319336
32 1 0.2511272430419922
33 1 0.2513439655303955
34 1 0.2500004768371582
35 1 0.2197577953338623
36 1 0.25242042541503906
37 1 0.25417590141296387
38 1 0.2530484199523926
39 1 0.2497425079345703
40 1 0.22060060501098633
41 1 0.

We can not prove the experimental divisiblity of bits 0, 16, 32, 48 with key-independent trail summation. So key-dependent trail summation is necessary

In [15]:
r, u, v, m = 3, bit_permutation(0xf), 0x1<<0, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 1 u's after 0.2366185188293457 seconds
computing propagation forwards
forwards propagation resulted in 4 u's after 0.5005602836608887 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.23797178268432617 seconds
computing propagation backwards
backwards propagation resulted in 4 v's after 0.5034835338592529 seconds
computing propagation forwards
forwards propagation resulted in 4 u's after 0.27051734924316406 seconds
computing propagation forwards
forwards propagation resulted in 4 u's after 0.26973795890808105 seconds
computing propagation forwards
forwards propagation resulted in 4 u's after 0.27056455612182617 seconds
{} 2.794898748397827


In [10]:
r, u, v, m = 3, bit_permutation(0xf), 0x1<<16, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 113 u's after 28.75857973098755 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.7615277767181396 seconds
computing propagation backwards
backwards propagation resulted in 6 v's after 1.8258044719696045 seconds
computing propagation backwards
backwards propagation resulted in 11 v's after 2.523365020751953 seconds
computing propagation backwards
backwards propagation resulted in 89 v's after 5.146726131439209 seconds
computing propagation backwards
backwards propagation resulted in 115 v's after 5.964597940444946 seconds
computing propagation forwards
forwards propagation resulted in 115 u's after 11.517974615097046 seconds
{} 58.32411813735962


In [11]:
r, u, v, m = 3, bit_permutation(0xf), 0x1<<32, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 113 u's after 29.33456826210022 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.7819900512695312 seconds
computing propagation backwards
backwards propagation resulted in 7 v's after 2.049804449081421 seconds
computing propagation backwards
backwards propagation resulted in 9 v's after 1.9225759506225586 seconds
computing propagation backwards
backwards propagation resulted in 45 v's after 4.382925987243652 seconds
computing propagation backwards
backwards propagation resulted in 57 v's after 3.307507038116455 seconds
computing propagation backwards
backwards propagation resulted in 113 v's after 9.761418581008911 seconds
{} 53.420326232910156


In [12]:
r, u, v, m = 3, bit_permutation(0xf), 0x1<<48, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 113 u's after 24.54600214958191 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.7759716510772705 seconds
computing propagation backwards
backwards propagation resulted in 6 v's after 1.0901219844818115 seconds
computing propagation backwards
backwards propagation resulted in 11 v's after 2.285433292388916 seconds
computing propagation backwards
backwards propagation resulted in 89 v's after 4.8846259117126465 seconds
computing propagation backwards
backwards propagation resulted in 115 v's after 5.346904277801514 seconds
computing propagation forwards
forwards propagation resulted in 115 u's after 10.399938583374023 seconds
{} 51.158610820770264


## 5 round property
input set is `0xfff0`

In [10]:
r, u = 4, bit_permutation(0xfff0)
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 5 1.5250649452209473
1 5 1.4689936637878418
2 5 1.515817642211914
3 5 1.5141234397888184
4 2 0.5004768371582031
5 2 0.5270683765411377
6 2 0.49581074714660645
7 2 0.5238075256347656
8 3 0.7477242946624756
9 3 0.7515132427215576
10 3 0.7556173801422119
11 3 0.7526369094848633
12 2 0.5289537906646729
13 2 0.49215173721313477
14 2 0.5213949680328369
15 2 0.5278725624084473
16 4 1.0674059391021729
17 4 1.0902478694915771
18 4 1.1044836044311523
19 4 1.0985925197601318
20 1 0.33086705207824707
21 1 0.3263704776763916
22 1 0.3270578384399414
23 1 0.3257408142089844
24 2 0.5274331569671631
25 2 0.49952125549316406
26 2 0.5248370170593262
27 2 0.4966287612915039
28 1 0.33116698265075684
29 1 0.32586669921875
30 1 0.326690673828125
31 1 0.326369047164917
32 4 1.0820369720458984
33 4 1.0728693008422852
34 4 1.0900442600250244
35 4 1.0883986949920654
36 2 0.49613404273986816
37 2 0.5261883735656738
38 2 0.49697327613830566
39 2 0.5292601585388184
40 2 0.4993584156036377
41 2 0.5267055034637451


lots of results that are not yet correct: 4, 12, 20, 28, 52, 60

In [16]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<4, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 218.02899837493896 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.35543036460876465 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.3599092960357666 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.35267019271850586 seconds
computing propagation backwards
backwards propagation resulted in 26 v's after 4.3697829246521 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.6577587127685547 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 4.903039216995239 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 14.447097063064575 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 255.76907539367676 seconds
{} 501.623961687088


In [18]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<12, 3
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 212.72796320915222 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.36662864685058594 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.3709089756011963 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.3612980842590332 seconds
computing propagation backwards
backwards propagation resulted in 16 v's after 2.697843551635742 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.4336442947387695 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 8.106675148010254 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 16.62109351158142 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 307.1584234237671 seconds
{} 550.7426400184631


In [19]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<20, 2
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 191.8865613937378 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.31305408477783203 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.3182713985443115 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.3183164596557617 seconds
computing propagation backwards
backwards propagation resulted in 26 v's after 3.825925827026367 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.838059902191162 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 4.592368125915527 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 14.292599201202393 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 223.516756772995 seconds
{} 441.7480754852295


In [20]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<28, 2
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 188.14611887931824 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.2992284297943115 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.2969856262207031 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.29407787322998047 seconds
computing propagation backwards
backwards propagation resulted in 16 v's after 2.288576364517212 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.248098373413086 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 6.962957382202148 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 14.230446577072144 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 269.1645841598511 seconds
{} 483.7348806858063


In [21]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<52, 2
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 181.21717429161072 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.2949984073638916 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.2968924045562744 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.2952091693878174 seconds
computing propagation backwards
backwards propagation resulted in 26 v's after 3.652731418609619 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.6618285179138184 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 4.5985846519470215 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 12.369237184524536 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 270.33962059020996 seconds
{} 475.53065490722656


In [22]:
r, u, v, m = 4, bit_permutation(0xfff0), 0x1<<60, 2
stime = time()
rfs, cor_evals = multiply_out_precursor_set(r, u)
res = compute_exact_correlation_mod(rfs, cor_evals, 0, v, m, precursor = False, NThreads = NThreads, debug=True)
print(res, time()-stime)

computing propagation forwards
forwards propagation resulted in 883 u's after 178.58017468452454 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.23790931701660156 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.23991680145263672 seconds
computing propagation backwards
backwards propagation resulted in 1 v's after 0.23831605911254883 seconds
computing propagation backwards
backwards propagation resulted in 16 v's after 2.2912025451660156 seconds
computing propagation backwards
backwards propagation resulted in 36 v's after 1.1874792575836182 seconds
computing propagation backwards
backwards propagation resulted in 198 v's after 6.924335241317749 seconds
computing propagation backwards
backwards propagation resulted in 276 v's after 12.565388441085815 seconds
computing propagation backwards
backwards propagation resulted in 883 v's after 218.20102262496948 seconds
{} 421.25068283081055


## 6 round property
input set is `0xffffffff`

In [11]:
r, u = 4, bit_permutation(bit_permutation(0xffffffff))
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 7 2.48724365234375
1 4 0.9361240863800049
2 4 0.9522249698638916
3 4 0.940030574798584
4 4 0.968026876449585
5 1 0.2955169677734375
6 1 0.28803110122680664
7 1 0.28618311882019043
8 4 0.9307601451873779
9 2 0.4635324478149414
10 2 0.4350442886352539
11 2 0.4326474666595459
12 4 0.9585285186767578
13 1 0.29141998291015625
14 1 0.28740978240966797
15 1 0.2931101322174072
16 7 2.596034288406372
17 4 0.9697849750518799
18 4 0.9659574031829834
19 4 0.9382107257843018
20 4 0.9842464923858643
21 1 0.2590000629425049
22 1 0.2906985282897949
23 1 0.28949642181396484
24 4 0.9756455421447754
25 2 0.4388422966003418
26 2 0.4723489284515381
27 2 0.43952465057373047
28 4 0.9598736763000488
29 1 0.29174280166625977
30 1 0.28904223442077637
31 1 0.28838014602661133
32 7 2.700803756713867
33 4 0.9655857086181641
34 4 0.9359400272369385
35 4 0.9640052318572998
36 4 0.9339613914489746
37 1 0.29046130180358887
38 1 0.2891383171081543
39 1 0.28849172592163086
40 4 0.9253647327423096
41 2 0.47651958465576

## 7 round property
input set is `0xfffffffffffff000`

In [12]:
r, u = 6, bit_permutation(0xfffffffffffff000)
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 9 167.35044884681702
1 5 4.3751702308654785
2 5 3.8621418476104736
3 5 4.214468479156494
4 4 4.0134100914001465
5 2 0.6675982475280762
6 2 0.6776266098022461
7 2 0.787452220916748
8 5 5.204667091369629
9 2 0.6398227214813232
10 2 0.5867519378662109
11 2 0.630108118057251
12 4 3.9711291790008545
13 2 0.7191829681396484
14 2 0.640343427658081
15 2 0.7879593372344971
16 8 157.85045909881592
17 4 3.850558042526245
18 4 2.923154354095459
19 4 3.6711604595184326
20 3 3.2654433250427246
21 1 0.49759745597839355
22 1 0.4195077419281006
23 1 0.4559352397918701
24 4 4.660343647003174
25 1 0.44526171684265137
26 1 0.45801210403442383
27 1 0.4613823890686035
28 3 2.8436996936798096
29 1 0.46830034255981445
30 1 0.5298399925231934
31 1 0.4577972888946533
32 8 34.54297161102295
33 4 3.122617483139038
34 4 2.4244155883789062
35 4 2.9879932403564453
36 5 17.0371196269989
37 1 0.47055745124816895
38 1 0.4377262592315674
39 1 0.44525766372680664
40 5 8.63122034072876
41 2 0.7962989807128906
42 2 0.746

## 8 round property
input set is `0xfffffffffffffffe`

In [13]:
r, u = 8, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 8 330.57228684425354
1 5 15.132317543029785
2 5 17.322696685791016
3 5 24.815342903137207
4 5 31.44292140007019
5 2 1.783715009689331
6 2 1.2431809902191162
7 2 1.4194507598876953
8 5 15.615493297576904
9 3 2.612935781478882
10 3 2.4056050777435303
11 3 2.1462252140045166
12 5 27.115337133407593
13 2 1.2138972282409668
14 2 1.510030746459961
15 2 1.5824542045593262
16 7 548.4741897583008
17 4 14.619363069534302
18 5 41.995516538619995
19 4 11.290764570236206
20 4 35.61488914489746
21 2 1.3464977741241455
22 2 1.4534215927124023
23 2 1.0089869499206543
24 5 41.82558751106262
25 2 1.2331511974334717
26 2 1.8038866519927979
27 2 0.9913780689239502
28 4 34.11062216758728
29 2 1.1908729076385498
30 2 1.0687317848205566
31 2 1.2806901931762695
32 7 202.52908945083618
33 4 6.610492467880249
34 5 21.89538812637329
35 4 9.656897068023682
36 5 44.05936622619629
37 2 1.0076732635498047
38 2 1.2469794750213623
39 2 1.2623717784881592
40 5 24.762900352478027
41 3 3.4211204051971436
42 3 3.3569943

## 9 round property
input set is `0xfffffffffffffffe`

In [14]:
r, u = 9, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    print(i, get_divisibility_no_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i), time()-start)

0 2 8.959420442581177
1 1 0.7922048568725586
2 1 1.1065921783447266
3 1 0.6398978233337402
4 2 4.5299413204193115
5 0 0.3953526020050049
6 0 0.36408472061157227
7 0 0.3621094226837158
8 2 3.3394381999969482
9 0 0.3856017589569092
10 0 0.38556718826293945
11 0 0.36624956130981445
12 2 4.179787635803223
13 0 0.3673231601715088
14 0 0.3672499656677246
15 0 0.39612770080566406
16 1 2.32179856300354
17 1 1.0822093486785889
18 1 0.8791511058807373
19 1 1.0459315776824951
20 1 1.5157625675201416
21 0 0.35628652572631836
22 0 0.35953450202941895
23 0 0.35550689697265625
24 1 1.0494945049285889
25 0 0.36194348335266113
26 0 0.3554267883300781
27 0 0.3563525676727295
28 1 1.6621088981628418
29 0 0.3569784164428711
30 0 0.36039113998413086
31 0 0.35416197776794434
32 1 0.967618465423584
33 1 1.340033769607544
34 1 1.0420067310333252
35 1 0.8070616722106934
36 1 1.3918180465698242
37 0 0.36101460456848145
38 0 0.35378479957580566
39 0 0.350100040435791
40 1 0.8163673877716064
41 0 0.35925865173339

# Zero correlation

In [46]:
# output mask set of size $2^{16}$
from pysat.card import CardEnc, EncType
r, u = 5, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(64):
    start = time()
    new_output_vars = output_vars[:i] + output_vars[i+1:]
    new_count_vars = count_vars + output_vars[i:i+1]
    print(i, 64-get_divisibility_no_trail(model, input_vars, new_output_vars, key_vars, new_count_vars, 0xffffffffffffffff, 0, precursor=False), time()-start)

0 -inf 0.12193179130554199
1 -inf 0.042684316635131836
2 -inf 0.0427250862121582
3 -inf 0.0422208309173584
4 -inf 0.04222750663757324
5 -inf 0.09306550025939941
6 -inf 0.042958974838256836
7 -inf 0.04239916801452637
8 -inf 0.04230308532714844
9 -inf 0.04217362403869629
10 -inf 0.0908362865447998
11 -inf 0.042249202728271484
12 -inf 0.04255342483520508
13 -inf 0.042409658432006836
14 -inf 0.04232430458068848
15 -inf 0.0906682014465332
16 -inf 0.04257798194885254
17 -inf 0.04207301139831543
18 -inf 0.042078495025634766
19 -inf 0.042479515075683594
20 -inf 0.0912024974822998
21 -inf 0.04260754585266113
22 -inf 0.042386770248413086
23 -inf 0.04231572151184082
24 -inf 0.04196882247924805
25 -inf 0.04268789291381836
26 -inf 0.08939480781555176
27 -inf 0.04287838935852051
28 -inf 0.0424044132232666
29 -inf 0.042643070220947266
30 -inf 0.04244709014892578
31 -inf 0.08908677101135254
32 -inf 0.04308032989501953
33 -inf 0.042993783950805664
34 -inf 0.042540788650512695
35 -inf 0.0421166419982910

In [37]:
# output mask set of size $2^{16}$
from pysat.card import CardEnc, EncType
r, u = 5, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(4):
    start = time()
    new_output_vars = sum([output_vars[j::4] for j in range(4) if j != i], tuple())
    new_count_vars = count_vars + output_vars[i::4]
    print(i, 64-get_divisibility_no_trail(model, input_vars, new_output_vars, key_vars, new_count_vars, 0xffffffffffffffff, 0, precursor=False), time()-start)

0 -inf 0.042646169662475586
1 -inf 0.04218482971191406
2 -inf 0.04204869270324707
3 -inf 0.04234957695007324


In [38]:
# output mask set of size $2^{32}$
from pysat.card import CardEnc, EncType
r, u = 5, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(4):
    for j in range(i+1, 4):
        start = time()
        new_output_vars = sum([output_vars[k::4] for k in range(4) if k != i and k != j], tuple())
        new_count_vars = count_vars + output_vars[i::4] + output_vars[j::4]
        print(i, j,  64-get_divisibility_no_trail(model, input_vars, new_output_vars, key_vars, new_count_vars, 0xffffffffffffffff, 0, precursor=False), time()-start)

0 1 -inf 0.09158682823181152
0 2 41 32.26926064491272
0 3 -inf 0.04246401786804199
1 2 -inf 0.09478640556335449
1 3 -inf 0.04185628890991211
2 3 -inf 0.04194307327270508


In [36]:
# output mask set of size $2^{48}$
from pysat.card import CardEnc, EncType
r, u = 5, 0xfffffffffffffffe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
for i in range(4):
    start = time()
    new_output_vars = output_vars[i::4]
    new_count_vars = count_vars + sum([output_vars[j::4] for j in range(4) if j != i], tuple())
    print(i, 64-get_divisibility_no_trail(model, input_vars, new_output_vars, key_vars, new_count_vars, 0xffffffffffffffff, 0, precursor=False), time()-start)

0 -inf 0.04249691963195801
1 41 41.78449010848999
2 -inf 0.042351484298706055
3 41 39.716124534606934


In [58]:
# 6 round property
v = 11
M = UT_matrix(PRESENT_sbox, 4, 4)
res = M[0, :]
for u in range(1, 16):
    if u | v == v:
        res += (-2)**u.bit_count()*M[u, :]
print(res)

[-1  0  0  0  2  0  0  0  2  0 -2  0 -2  0  0  0]


# Improving key recovery

## 4 Rounds

In [176]:
r, u = 4, 0xe
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[1, 1, 1, 1]


In [177]:
r, u = 4, 0xf
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[3, 2, 2, 2]


In [178]:
r, u = 4, 0x1000f
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[3, 3, 3, 3]


## 5 rounds

In [179]:
r, u = 5, 0xf
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[1, 1, 1, 1]


In [180]:
r, u = 5, 0x1000f
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[2, 1, 1, 1]


In [181]:
r, u = 5, 0xf000f
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[4, 1, 1, 1]


## 6 rounds

In [182]:
for i in range(16):
    r, u = 6, 0xff0 | (i << 12)
    rfs = multiply_out_precursor_set(r, u)[0][1::2]
    key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
    f = construct_iterated_cipher(rfs, key_masks)
    model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
    res = {}
    print(hex(u), [get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

0xff0 [1, 0, 0, 0]
0x1ff0 [1, 0, 0, 0]
0x2ff0 [1, 0, 0, 0]
0x3ff0 [1, 0, 0, 0]
0x4ff0 [1, 0, 0, 0]
0x5ff0 [1, 0, 0, 0]
0x6ff0 [1, 0, 0, 0]
0x7ff0 [1, 0, 0, 0]
0x8ff0 [1, 0, 0, 0]
0x9ff0 [1, 0, 0, 0]
0xaff0 [1, 0, 0, 0]
0xbff0 [1, 0, 0, 0]
0xcff0 [1, 0, 0, 0]
0xdff0 [1, 0, 0, 0]
0xeff0 [2, 0, 0, 0]
0xfff0 [4, 1, 2, 1]


## 7 rounds

In [183]:
r, u = 7, 0xffff
rfs = multiply_out_precursor_set(r, u)[0][1::2]
key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
f = construct_iterated_cipher(rfs, key_masks)
model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
res = {}
print([get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

[1, 0, 0, 0]


## 8 rounds

In [184]:
for i in range(16):
    r, u = 8, 0xffffffffffff0000 | i
    rfs = multiply_out_precursor_set(r, u)[0][1::2]
    key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
    f = construct_iterated_cipher(rfs, key_masks)
    model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
    res = {}
    print(hex(u), [get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

0xffffffffffff0000 [1, 0, 0, 0]
0xffffffffffff0001 [1, 0, 0, 0]
0xffffffffffff0002 [1, 0, 0, 0]
0xffffffffffff0003 [2, 0, 1, 0]
0xffffffffffff0004 [1, 0, 0, 0]
0xffffffffffff0005 [2, 0, 1, 0]
0xffffffffffff0006 [1, 0, 0, 0]
0xffffffffffff0007 [2, 0, 1, 0]
0xffffffffffff0008 [1, 0, 0, 0]
0xffffffffffff0009 [2, 0, 1, 0]
0xffffffffffff000a [2, 0, 1, 0]
0xffffffffffff000b [2, 0, 1, 0]
0xffffffffffff000c [2, 0, 1, 0]
0xffffffffffff000d [2, 0, 1, 0]
0xffffffffffff000e [2, 0, 1, 0]
0xffffffffffff000f [2, 1, 1, 1]


## 9 rounds

In [185]:
for i in range(15):
    r, u = 9, 0xfffffffffffffff0 | i
    rfs = multiply_out_precursor_set(r, u)[0][1::2]
    key_masks = [2**(rfs[i].input_size)-1 for i in range(r)] + [2**(rfs[-1].output_size)-1]
    f = construct_iterated_cipher(rfs, key_masks)
    model, input_vars, output_vars, key_vars, count_vars = f.to_UT_model()
    res = {}
    print(hex(u), [get_divisibility_no_key_dependent_trail(model, input_vars, output_vars, key_vars, count_vars, 0, 1<<i) for i in range(4)])

0xfffffffffffffff0 [1, 0, 0, 0]
0xfffffffffffffff1 [1, 0, 0, 0]
0xfffffffffffffff2 [1, 0, 0, 0]
0xfffffffffffffff3 [1, 0, 0, 0]
0xfffffffffffffff4 [1, 0, 0, 0]
0xfffffffffffffff5 [1, 0, 0, 0]
0xfffffffffffffff6 [1, 0, 0, 0]
0xfffffffffffffff7 [1, 0, 0, 0]
0xfffffffffffffff8 [1, 0, 0, 0]
0xfffffffffffffff9 [1, 0, 0, 0]
0xfffffffffffffffa [1, 0, 0, 0]
0xfffffffffffffffb [1, 0, 0, 0]
0xfffffffffffffffc [1, 0, 0, 0]
0xfffffffffffffffd [1, 0, 0, 0]
0xfffffffffffffffe [2, 1, 1, 1]
